In [6]:
import numpy as np
import pandas as pd
import wandb
import datetime
from keras.datasets import fashion_mnist

In [7]:
wandb.login()

True

In [8]:
# Load the Fashion MNIST dataset
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

x_train_flat = x_train.reshape(x_train.shape[0], -1)
train_df = pd.DataFrame(x_train_flat)
train_df['label'] = y_train
train_df['label_name'] = [class_names[label] for label in y_train]

x_test_flat = x_test.reshape(x_test.shape[0], -1)
test_df = pd.DataFrame(x_test_flat)
test_df['label'] = y_test
test_df['label_name'] = [class_names[label] for label in y_test]

In [18]:
print(f"Training set shape: {train_df.shape}")
print(f"Test set shape: {test_df.shape}")

Training set shape: (60000, 786)
Test set shape: (10000, 786)


In [22]:
# Initialize a W&B run
wandb.init(
    entity="bullseye2608-indian-institute-of-technology-madras",
    
    project="my-awesome-project", 
    
    name="ImageExamples_"+datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    )

# Log images to W&B
wandb.log({
    "fashion_mnist_samples": [
        wandb.Image(x_train_flat[np.where(y_train == i)[0][0]].reshape(28,28), caption=class_names[i])
        for i in range(10)
    ]
})

wandb.finish()


In [71]:
def initialise_neural_network(input_size=784, num_hidden_layers=1, num_neurons=32, output_size=10):
    assert num_hidden_layers > 0
    assert num_neurons > 0
    next_size = [input_size] + [num_neurons] * num_hidden_layers + [output_size]
    weights = []
    biases = []
    for i in range(1, len(next_size)):
        weights.append(np.random.uniform(-0.5, 0.5, (next_size[i], next_size[i-1])))
        biases.append(np.zeros(next_size[i]))
    
    return [*weights, *biases]

def sigmoid(x):
    # For positive values, use standard formula
    # For negative values, use a mathematically equivalent but more stable form
    mask = x >= 0
    result = np.zeros_like(x, dtype=float)
    
    # For positive inputs: 1/(1+exp(-x))
    result[mask] = 1 / (1 + np.exp(-x[mask]))
    
    # For negative inputs: exp(x)/(1+exp(x))
    # This avoids computing exp of large positive numbers
    exp_x = np.exp(x[~mask])
    result[~mask] = exp_x / (1 + exp_x)
    
    return result

def softmax(x, tau=1.0):
    # Subtract the max value to avoid overflow
    x = x - np.max(x)
    exp_x = np.exp(x / tau)
    return exp_x / np.sum(exp_x)

def cross_entropy_loss(predictions, targets):
    return -np.sum(targets * np.log(predictions))

def cross_entropy_loss_gradient(predictions, targets):
    return predictions - targets

def forward_propagation(input_data, theta, tau):
    L = len(theta) // 2
    a, h = [], []
    h_temp = input_data
    
    for i in range(1,L):
        a_temp = np.dot(theta[i-1], h_temp) + theta[L+i-1]
        h_temp = sigmoid(a_temp)
        a.append(a_temp)
        h.append(h_temp)
    
    a_temp = np.dot(theta[L-1], h[-1]) + theta[-1]
    h_temp = softmax(a_temp, tau)
    a.append(a_temp)
    h.append(h_temp)
    
    return a, h

def predict_neural_network(input_data, theta, tau):
    L = len(theta) // 2
    weights = theta[:L]
    biases = theta[L:]
    x = input_data
    for i in range(len(weights)):
        x = np.dot(weights[i], x) + biases[i]
        x = sigmoid(x)
        
    x = softmax(x, tau)
    return x

In [83]:
theta = initialise_neural_network(input_size=784, num_hidden_layers=2, num_neurons=32, output_size=10)
a, h = forward_propagation(x_train_flat[0], theta, tau=1.0)
h[-1]

array([0.06332199, 0.15629137, 0.03677306, 0.03761336, 0.07812488,
       0.05733909, 0.0710442 , 0.10348283, 0.15402127, 0.24198796])

In [66]:
theta[0].shape

(32, 784)

In [52]:
a = np.array([1,2,3])
a

array([1, 2, 3])

In [55]:
# add an element to a
a = np.append(a, 4)

In [56]:
a

array([1, 2, 3, 4])